In [ ]:
!pip install pandas
!pip install numpy

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split 

In [ ]:
df = pd.read_csv('/Users/aryan/Desktop/CS7641_Group58/train_dataset_preprocessed.csv')

In [ ]:
#split data into features/target

x = df[['odds_ft_1', 'odds_ft_x', 'odds_ft_2', 'position_a_prematch', 'position_b_prematch']]
y = df['Outcome']

In [ ]:
#train test split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
#standardized features needed for svm - alr did in preprocessing

svm = SVC(kernel='linear', C = 1.0, random_state = 42) #init model

rf = RandomForestClassifier(n_estimators=300, random_state=42)

log_reg = LogisticRegression(multi_class='multinomial', max_iter=1000)

gbc = GradientBoostingClassifier(
    n_estimators = 100,
    max_depth=50,
    learning_rate=0.1,
    random_state=42
)


In [ ]:
#applying PCA prior to fitting model, reducing dimensionality as 39 features is a lot
pca = PCA(n_components = 0.90)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
#train model with train data
rf.fit(x_train, y_train)

In [ ]:
#predictions on test
preds = rf.predict(x_test)

In [ ]:
#evaluate the predictions

acc = accuracy_score(preds, y_test)
cm = confusion_matrix(preds, y_test)
class_report = classification_report(preds, y_test)
acc
class_report


In [ ]:
acc

In [ ]:
cm

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', None]
}

# Initialize Random Forest
rf = RandomForestClassifier(random_state=42)

# Perform GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)